### 1. Reading input data

In [2]:
import pyodbc 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

In [25]:
import sqlalchemy as sa
from sqlalchemy import create_engine, event
from sqlalchemy.engine.url import URL

## Reading WILDFIRE data - mother table:
print ("connect to engine......")
engine = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Climate_Impact" + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')

print ("connection to greenmonkey found")
print ("Reading wilfdire table from greenmonkey")
query=('''
SELECT *
  FROM [Climate_Impact].[wild_fire].[wild_fires_20002022_v4_mother]
''')

wildfire = pd.read_sql(query, engine)
print(wildfire.head())

connect to engine......
connection to greenmonkey found
Reading wilfdire table from greenmonkey
   ADM_ID  ProtArea envZones   lcf3 clc0018   GridNum10km  y01012000  \
0    1446       NaN   Boreal     NC  324324  1.930579e+16          0   
1    1446       NaN   Boreal     NC  312312  2.364984e+16          0   
2    1446       NaN   Boreal  lcf53  412211  2.365057e+16          0   
3    1446       NaN   Boreal     NC  313313  2.365072e+16          0   
4    1446       NaN   Boreal     NC  313313  2.365080e+16          0   

   y01022000  y01032000  y01042000  ...  y01042022  y01052022  y01062022  \
0          0          0          0  ...          0          0          0   
1          0          0          0  ...          0          0          0   
2          0          0          0  ...          0          0          0   
3          0          0          0  ...          0          0          0   
4          0          0          0  ...          0          0          0   

   y01072022  

In [3]:
print(wildfire.head())

   ADM_ID  ProtArea envZones   lcf3 clc0018   GridNum10km  y01012000  \
0    1446       NaN   Boreal     NC  324324  1.930579e+16          0   
1    1446       NaN   Boreal     NC  312312  2.364984e+16          0   
2    1446       NaN   Boreal  lcf53  412211  2.365057e+16          0   
3    1446       NaN   Boreal     NC  313313  2.365072e+16          0   
4    1446       NaN   Boreal     NC  313313  2.365080e+16          0   

   y01022000  y01032000  y01042000  ...  y01042022  y01052022  y01062022  \
0          0          0          0  ...          0          0          0   
1          0          0          0  ...          0          0          0   
2          0          0          0  ...          0          0          0   
3          0          0          0  ...          0          0          0   
4          0          0          0  ...          0          0          0   

   y01072022  y01082022  y01092022  y01102022  y01112022  y01122022  AreaHa  
0          0          0         

In [4]:
wf_cols= wildfire.columns[:-2] 
wf_LTA = wildfire[wf_cols] #wildfire Long Term Annual data
wf_LTA.head()

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01022022,y01032022,y01042022,y01052022,y01062022,y01072022,y01082022,y01092022,y01102022,y01112022
0,1446,NaN,Boreal,NC,324324,1.930579e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1446,NaN,Boreal,NC,312312,2.364984e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1446,NaN,Boreal,lcf53,412211,2.365057e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1446,NaN,Boreal,NC,313313,2.365072e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1446,NaN,Boreal,NC,313313,2.365080e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
wf19 = wf_LTA[wf_LTA['TN_0010']==19]

In [5]:
cols_0010 = wf_LTA.columns[6:138]
wf_LTA['BA_0010'] = wf_LTA[cols_0010].sum(axis=1) #sum of BurntArea in hectares from 00 to 10
cols_1121 = wf_LTA.columns[138:270]
wf_LTA['BA_1121'] = wf_LTA[cols_1121].sum(axis=1) #sum of BurntArea in hectares from 11 to 21
wf_LTA['TN_0010']= wf_LTA[wf_LTA.iloc[:, 6:138] > 0].count(axis='columns') # total number of wildfires for the period
wf_LTA['TN_1121']= wf_LTA[wf_LTA.iloc[:, 138:270] > 0].count(axis='columns') # total number of wildfires for the period
wf_LTA = wf_LTA.loc[:, ['ADM_ID','ProtArea','envZones', 'lcf3', 'clc0018', 'GridNum10km', 'BA_0010', 'BA_1121', 'TN_0010', 'TN_1121','AreaHa']]
wf_LTA['LTAA_0010'] = wf_LTA['BA_0010'].apply(lambda x: x/11).round(1) #Long Term Average Area  - divide area / n of years (11 for this period)
wf_LTA['LTAA_1121'] = wf_LTA['BA_1121'].apply(lambda x: x/11).round(1) #Long Term Average Area - divide area / n of years (11 for this period)
wf_LTA['LTAN_0010']=  wf_LTA['TN_0010'].apply(lambda x: x/11).round(1) # total number of wildfires divided by 11 years
wf_LTA['LTAN_1121']=  wf_LTA['TN_1121'].apply(lambda x: x/11).round(1) # total number of wildfires divided by 11 years
wf_LTA.head(10)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


KeyboardInterrupt: 

In [3]:
import sqlalchemy as sa
from sqlalchemy import create_engine, event
from sqlalchemy.engine.url import URL

## Reading WILDFIRE data at 10km:
print ("connect to engine......")
engine = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Climate_Impact" + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')

print ("connection to greenmonkey found")
print ("Reading wilfdire table from greenmonkey")
query=('''
SELECT     
      [GridNum10km]   
      ,[fire_date]
      ,sum([burned_area]) as [burned_area]
  FROM [Climate_Impact].[wild_fire].[wild_fires_2000_2022_light]
      
group by 
    [GridNum10km]
    ,[fire_date]
''')

df_wildfire10km = pd.read_sql(query, engine)
print(df_wildfire10km.head())

connect to engine......
connection to greenmonkey found
Reading wilfdire table from greenmonkey
    GridNum10km  fire_date  burned_area
0  2.311768e+16 2008-08-01           55
1  2.308264e+16 2017-07-01          635
2  9.713537e+15 2000-01-01          174
3  1.842461e+16 2000-01-01           32
4  9.732280e+15 2000-01-01          292


In [21]:
ttdata = df_wildfire10km[(df_wildfire10km['GridNum10km']==9712437179711488) and (df_wildfire10km['GridNum10km'] == 9712218136379392)]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [8]:
ttdata = df_wildfire10km[df_wildfire10km['GridNum10km']==9712437179711488]

In [9]:
ttdata.sort_values(by=['fire_date'])

,GridNum10km,fire_date,burned_area
37874,9.712437e+15,2000-01-01,2192
29187,9.712437e+15,2001-01-01,436
32521,9.712437e+15,2002-01-01,793
39256,9.712437e+15,2003-01-01,98
20687,9.712437e+15,2005-01-01,258
13866,9.712437e+15,2006-09-01,194
15166,9.712437e+15,2011-10-01,111
22662,9.712437e+15,2012-03-01,48
5353,9.712437e+15,2013-06-01,30
41854,9.712437e+15,2013-08-01,105


In [26]:
ttdata[['GridNum10km','Year']].groupby('Year').count()

year
2000    1
2001    1
2002    1
2003    1
2005    1
2006    1
2011    1
2012    1
2013    2
2016    1
2017    5
2019    1
2020    1
2022    1
dtype: int64

In [36]:
#ttdata.groupby([ttdata.GridNum10km, ttdata.fire_date.dt.year.rename('year')]).size().reset_index()
ttdata['count']=ttdata.groupby([ttdata.GridNum10km, ttdata.fire_date.dt.year.rename('year')]).size()

TypeError: incompatible index of inserted column with frame index

In [37]:
ttdata.head()

,GridNum10km,fire_date,burned_area
1477,9.712437e+15,2013-06-01,30
2765,9.712437e+15,2017-07-01,2531
3242,9.712437e+15,2017-10-01,66
5249,9.712437e+15,2017-11-01,30
11215,9.712437e+15,2012-03-01,48


In [41]:
ttdata['Count_d'] = ttdata.groupby('GridNum10km', 'fire_date')['fire_date'].transform('size')

'''y = ttdata['fire_date'].dt.year
m = df['Date'].dt.month

df['Count_d'] = df.groupby('Date')['Date'].transform('size')
df['Count_m'] = df.groupby([y, m])['Date'].transform('size')
df['Count_y'] = df.groupby(y)['Date'].transform('size')'''

ValueError: No axis named fire_date for object type DataFrame

In [42]:
ttdata.groupby('GridNum10km', 'fire_date').sum()

ValueError: No axis named fire_date for object type DataFrame

In [32]:
ttdata.groupby([ttdata.fire_date.dt.year.rename('year')]).agg('count')

,GridNum10km,fire_date,burned_area
year,,,
2000,1,1,1
2001,1,1,1
2002,1,1,1
2003,1,1,1
2005,1,1,1
2006,1,1,1
2011,1,1,1
2012,1,1,1
2013,2,2,2


In [43]:
ttdata.groupby([ttdata.fire_date.dt.year.rename('year')]).size()

year
2000    1
2001    1
2002    1
2003    1
2005    1
2006    1
2011    1
2012    1
2013    2
2016    1
2017    5
2019    1
2020    1
2022    1
dtype: int64

In [44]:
ttdata.groupby([ttdata.fire_date.dt.year.rename('year'),'GridNum10km']).size()

year  GridNum10km 
2000  9.712437e+15    1
2001  9.712437e+15    1
2002  9.712437e+15    1
2003  9.712437e+15    1
2005  9.712437e+15    1
2006  9.712437e+15    1
2011  9.712437e+15    1
2012  9.712437e+15    1
2013  9.712437e+15    2
2016  9.712437e+15    1
2017  9.712437e+15    5
2019  9.712437e+15    1
2020  9.712437e+15    1
2022  9.712437e+15    1
dtype: int64

In [52]:
ttdata.groupby([ttdata.fire_date.dt.year.rename('year'),'GridNum10km']).size().reset_index(name='fire_n')

,year,GridNum10km,fire_n
0,2000,9.712437e+15,1
1,2001,9.712437e+15,1
2,2002,9.712437e+15,1
3,2003,9.712437e+15,1
4,2005,9.712437e+15,1
5,2006,9.712437e+15,1
6,2011,9.712437e+15,1
7,2012,9.712437e+15,1
8,2013,9.712437e+15,2
9,2016,9.712437e+15,1


In [64]:
sumDataFrame = (ttdata[ttdata.fire_date.dt.year.between(2000, 2010)]
                  .groupby(['GridNum10km', ttdata.fire_date.dt.year])
                  .agg(sum))

In [61]:
sumDataFrame1121 = (ttdata[ttdata.fire_date.dt.year.between(2011, 2021)]
                  .groupby(['GridNum10km', ttdata.fire_date.dt.year])
                  .agg(sum))

In [60]:
sumDataFrame

burned_area
GridNum10km  fire_date             
9.712437e+15 2000              2192
             2001               436
             2002               793
             2003                98
             2005               258
             2006               194

In [62]:
sumDataFrame1121

burned_area
GridNum10km  fire_date             
9.712437e+15 2011               111
             2012                48
             2013               135
             2016               205
             2017              2865
             2019               322
             2020               757

In [66]:
sizeDataFrame = (ttdata[ttdata.fire_date.dt.year.between(2000, 2010)]
                  .groupby(['GridNum10km', ttdata.fire_date.dt.year])
                  .size().reset_index(name='fire_n'))
sizeDataFrame

,GridNum10km,fire_date,fire_n
0,9.712437e+15,2000,1
1,9.712437e+15,2001,1
2,9.712437e+15,2002,1
3,9.712437e+15,2003,1
4,9.712437e+15,2005,1
5,9.712437e+15,2006,1


In [72]:
sizeDataFrame.groupby(['GridNum10km']).size().reset_index(name='LTN')

,GridNum10km,LTN
0,9.712437e+15,6


In [74]:
(ttdata[ttdata.fire_date.dt.year.between(2000, 2010)]
                  .groupby(['GridNum10km', ttdata.fire_date.dt.year])
                  .size()).groupby(['GridNum10km']).size().reset_index(name='fire_n_0010')

,GridNum10km,fire_n_0010
0,9.712437e+15,6


In [4]:
fire_n_0010 = (df_wildfire10km[df_wildfire10km.fire_date.dt.year.between(2000, 2010)]
                  .groupby(['GridNum10km', df_wildfire10km.fire_date.dt.year])
                  .size()).groupby(['GridNum10km']).size().reset_index(name='fire_n_0010')

In [76]:
fire_n_0010

,GridNum10km,fire_n_0010
0,4.637216e+15,1
1,4.637220e+15,1
2,4.637280e+15,1
3,4.637285e+15,1
4,4.637289e+15,1
...,...,...
6733,3.212697e+16,1
6734,3.212712e+16,1
6735,3.212718e+16,1
6736,3.212883e+16,1


In [77]:
fire_n_0010[fire_n_0010['GridNum10km']==9712437179711488]

,GridNum10km,fire_n_0010
783,9.712437e+15,6


In [10]:
fire_n_0010.fire_n_0010.max()

10

In [11]:
fire_n_0010[fire_n_0010['fire_n_0010']==10]

,GridNum10km,fire_n_0010
740,9.712149e+15,10
745,9.712197e+15,10
755,9.712265e+15,10
822,9.713120e+15,10
836,9.713232e+15,10
846,9.713300e+15,10
2659,1.840816e+16,10


In [5]:
fire_n_1121 = (df_wildfire10km[df_wildfire10km.fire_date.dt.year.between(2011, 2021)]
                  .groupby(['GridNum10km', df_wildfire10km.fire_date.dt.year])
                  .size()).groupby(['GridNum10km']).size().reset_index(name='fire_n_1121')

In [6]:
fire_n_1121[fire_n_1121['GridNum10km']==9712437179711488]

,GridNum10km,fire_n_1121
705,9.712437e+15,7


In [12]:
fire_n_1121.fire_n_1121.max()

11

In [13]:
fire_n_1121[fire_n_1121['fire_n_1121']==11]

,GridNum10km,fire_n_1121
653,9.712081e+15,11
740,9.713107e+15,11
750,9.713176e+15,11
757,9.713232e+15,11
3002,1.840744e+16,11


In [14]:
fire_n_0022 = (df_wildfire10km[df_wildfire10km.fire_date.dt.year.between(2000, 2022)]
                  .groupby(['GridNum10km', df_wildfire10km.fire_date.dt.year])
                  .size()).groupby(['GridNum10km']).size().reset_index(name='fire_n_0022')

In [15]:
fire_n_0022.fire_n_0022.max()

22

In [20]:
fire_n_0022[fire_n_0022['fire_n_0022']==22]

,GridNum10km,fire_n_0022
1071,9.713232e+15,22


In [24]:
fire_n_0022.sort_values(by=['fire_n_0022'], ascending=False).head(20)

,GridNum10km,fire_n_0022
1071,9.713232e+15,22
4601,1.840816e+16,21
1054,9.713107e+15,21
1057,9.713120e+15,21
4609,1.840823e+16,20
967,9.712089e+15,20
819,9.696146e+15,19
1061,9.713163e+15,19
1077,9.713258e+15,19
990,9.712265e+15,19


In [21]:
ttdata.sort_values(by=['fire_n_0022'], ascending=False)

KeyError: 'fire_n_0022'

In [ ]:
sumDataFrame1121 = (ttdata[ttdata.fire_date.dt.year.between(2011, 2021)]
                  .groupby(['GridNum10km', ttdata.fire_date.dt.year])
                  .agg(average))

In [45]:
ttdata['count']= ttdata.groupby([ttdata.fire_date.dt.year.rename('year'),'GridNum10km']).size()

TypeError: incompatible index of inserted column with frame index

In [ ]:
df['birthdate'].groupby([GridNum10km, df.birthdate.dt.month]).agg('count')

In [76]:
wf_LTA[(wf_LTA['ADM_ID']==1521) & (wf_LTA['GridNum10km'] == 9712437179711488) & (wf_LTA['clc0018'] == '321122')]

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,BA_0010,BA_1121,AreaHa,ABA_0010,ABA_1121
939379,1521,NaN,Lusitanian,lcf32,321122,9.712437e+15,4,2,10.0,0.0,0.0


In [77]:
wf_LTA[(wf_LTA['ADM_ID']==1521) & (wf_LTA['GridNum10km'] == 9712218136379392) & (wf_LTA['clc0018'] == '211211')]

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,BA_0010,BA_1121,AreaHa,ABA_0010,ABA_1121
1168166,1521,NaN,Lusitanian,NC,211211,9.712218e+15,21,103,280.0,2.0,9.0


In [79]:
wf_cols= wildfire.columns[:-2] 
wf_LTA2 = wildfire[wf_cols]

In [82]:
tt = wf_LTA2[(wf_LTA2['ADM_ID']==1521) & (wf_LTA2['GridNum10km'] == 9712218136379392) & (wf_LTA2['clc0018'] == '211211')]

In [84]:
tt['count1121'] = tt[cols_1121].count(axis=1)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [87]:
tt['count0010'] = tt[cols_0010].count(axis=1)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [112]:
tt

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01062022,y01072022,y01082022,y01092022,y01102022,y01112022,y01122022,AreaHa,count1121,count0010
1168166,1521,NaN,Lusitanian,NC,211211,9.712218e+15,11,0,0,0,...,0,0,0,0,0,0,0,280.0,132,132


In [122]:
#tt.loc[6:138, (tt> 0).all()]
#df[df.columns[df.iloc[0] == 'xyz']]
##tt[tt.columns[tt.iloc[cols_0010]>0]]
##cols_0010
tt.iloc[:, 6:10]


,y01012000,y01022000,y01032000,y01042000
1168166,11,0,0,0


In [130]:
tt.iloc[:, 6:138] >0

,y01012000,y01022000,y01032000,y01042000,y01052000,y01062000,y01072000,y01082000,y01092000,y01102000,...,y01032010,y01042010,y01052010,y01062010,y01072010,y01082010,y01092010,y01102010,y01112010,y01122010
1168166,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [135]:
tt['ct_0010']= tt[tt.iloc[:, 6:138] > 0].count(axis='columns')

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [138]:
tt

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01082022,y01092022,y01102022,y01112022,y01122022,AreaHa,count1121,count0010,ct_0010,ct_1121
1168166,1521,NaN,Lusitanian,NC,211211,9.712218e+15,11,0,0,0,...,0,0,0,0,0,280.0,132,132,3,2


In [137]:
tt['ct_1121'] = tt[tt.iloc[:, 138:270] > 0].count(axis='columns')

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [131]:
tt.iloc[:, 6:138] >0 
df.count(axis='columns')
df.loc[df['shield'] > 35] = 0

SyntaxError: invalid syntax (<ipython-input-131-daed16ae6fd0>, line 1)

In [127]:
#tt[tt.iloc[:, 6:10] > 0].count()
tt[tt.iloc[:, 6:10] > 0]

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01062022,y01072022,y01082022,y01092022,y01102022,y01112022,y01122022,AreaHa,count1121,count0010
1168166,NaN,NaN,NaN,NaN,NaN,NaN,11,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
tt

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01062022,y01072022,y01082022,y01092022,y01102022,y01112022,y01122022,AreaHa,count1121,count0010
1168166,1521,NaN,Lusitanian,NC,211211,9.712218e+15,11,0,0,0,...,0,0,0,0,0,0,0,280.0,132,132


In [98]:
tt.transpose()[240:]

,1168166
y01072019,0
y01082019,0
y01092019,0
y01102019,0
y01112019,0
y01122019,0
y01012020,0
y01022020,0
y01032020,0
y01042020,0


In [100]:
77+26

103

In [65]:
cols_1121 = wildfire.columns[138:270]
len(cols_1121.tolist())

132

In [66]:
cols_1121.tolist()

['y01012011',
 'y01022011',
 'y01032011',
 'y01042011',
 'y01052011',
 'y01062011',
 'y01072011',
 'y01082011',
 'y01092011',
 'y01102011',
 'y01112011',
 'y01122011',
 'y01012012',
 'y01022012',
 'y01032012',
 'y01042012',
 'y01052012',
 'y01062012',
 'y01072012',
 'y01082012',
 'y01092012',
 'y01102012',
 'y01112012',
 'y01122012',
 'y01012013',
 'y01022013',
 'y01032013',
 'y01042013',
 'y01052013',
 'y01062013',
 'y01072013',
 'y01082013',
 'y01092013',
 'y01102013',
 'y01112013',
 'y01122013',
 'y01012014',
 'y01022014',
 'y01032014',
 'y01042014',
 'y01052014',
 'y01062014',
 'y01072014',
 'y01082014',
 'y01092014',
 'y01102014',
 'y01112014',
 'y01122014',
 'y01012015',
 'y01022015',
 'y01032015',
 'y01042015',
 'y01052015',
 'y01062015',
 'y01072015',
 'y01082015',
 'y01092015',
 'y01102015',
 'y01112015',
 'y01122015',
 'y01012016',
 'y01022016',
 'y01032016',
 'y01042016',
 'y01052016',
 'y01062016',
 'y01072016',
 'y01082016',
 'y01092016',
 'y01102016',
 'y01112016',
 'y011

In [59]:
len(wildfire.columns.tolist())

285

In [64]:
138+132

270

In [57]:
len(cols_0010.tolist())


132

In [58]:
cols_0010.tolist()

['y01012000',
 'y01022000',
 'y01032000',
 'y01042000',
 'y01052000',
 'y01062000',
 'y01072000',
 'y01082000',
 'y01092000',
 'y01102000',
 'y01112000',
 'y01122000',
 'y01012001',
 'y01022001',
 'y01032001',
 'y01042001',
 'y01052001',
 'y01062001',
 'y01072001',
 'y01082001',
 'y01092001',
 'y01102001',
 'y01112001',
 'y01122001',
 'y01012002',
 'y01022002',
 'y01032002',
 'y01042002',
 'y01052002',
 'y01062002',
 'y01072002',
 'y01082002',
 'y01092002',
 'y01102002',
 'y01112002',
 'y01122002',
 'y01012003',
 'y01022003',
 'y01032003',
 'y01042003',
 'y01052003',
 'y01062003',
 'y01072003',
 'y01082003',
 'y01092003',
 'y01102003',
 'y01112003',
 'y01122003',
 'y01012004',
 'y01022004',
 'y01032004',
 'y01042004',
 'y01052004',
 'y01062004',
 'y01072004',
 'y01082004',
 'y01092004',
 'y01102004',
 'y01112004',
 'y01122004',
 'y01012005',
 'y01022005',
 'y01032005',
 'y01042005',
 'y01052005',
 'y01062005',
 'y01072005',
 'y01082005',
 'y01092005',
 'y01102005',
 'y01112005',
 'y011

In [ ]:
cols_1121 = wf_LTA.columns[138:-1]


In [43]:
132*2

264

In [55]:
11*12

132

In [23]:
wf_LTA[(wf_LTA['ADM_ID']==1521) & (wf_LTA['GridNum10km'] == 9712437179711488) & (wf_LTA['clc0018'] == 321122)]


,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,BA_0010,BA_1121,AreaHa,ABA_0010,ABA_1121


In [24]:
wildfire[(wildfire['ADM_ID']==1521) & (wildfire['GridNum10km'] == 9712437179711488) & (wildfire['clc0018'] == 321122)]

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01062022,y01072022,y01082022,y01092022,y01102022,y01112022,y01122022,AreaHa,BA_0010,BA_1121


In [25]:
wildfire[(wildfire['ADM_ID']==1521) &  (wildfire['clc0018'] == 321122)]

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01062022,y01072022,y01082022,y01092022,y01102022,y01112022,y01122022,AreaHa,BA_0010,BA_1121


In [26]:
wildfire[(wildfire['ADM_ID']==1521) ]

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01062022,y01072022,y01082022,y01092022,y01102022,y01112022,y01122022,AreaHa,BA_0010,BA_1121
163,1521,1.0,Mediterranean North,NC,243243,9.712557e+15,0,0,0,0,...,0,0,0,0,0,0,0,88.0,13,126.0
3349,1521,1.0,Lusitanian,lcf72,313324,9.712231e+15,0,0,0,0,...,0,0,7,0,0,0,0,161.0,29,204.0
3350,1521,NaN,Mediterranean North,NC,243243,9.712557e+15,2,0,0,0,...,0,0,0,0,0,0,0,447.0,8,498.0
5814,1521,10.0,Mediterranean North,NC,313313,9.712575e+15,0,0,0,0,...,0,0,0,0,0,0,0,37.0,0,37.0
6373,1521,NaN,Lusitanian,NC,512512,9.712222e+15,0,0,0,0,...,0,0,0,0,0,0,0,60.0,0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676306,1521,NaN,Lusitanian,NC,222222,9.712360e+15,0,0,0,0,...,0,0,0,0,0,0,0,53.0,12,65.0
1676604,1521,1.0,Lusitanian,NC,112112,9.712231e+15,0,0,0,0,...,0,0,0,0,0,0,0,26.0,0,26.0
1677083,1521,NaN,Mediterranean North,lcf72,312324,9.729608e+15,0,0,0,0,...,0,0,0,0,0,0,0,52.0,1,59.0
1677447,1521,NaN,Mediterranean North,lcf61,321324,9.729608e+15,0,0,0,0,...,0,0,0,0,0,0,0,9.0,0,9.0


In [27]:
wildfire[(wildfire['ADM_ID']==1521) & (wildfire['GridNum10km'] == 9712437179711488) ]

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01062022,y01072022,y01082022,y01092022,y01102022,y01112022,y01122022,AreaHa,BA_0010,BA_1121
9316,1521,NaN,Lusitanian,lcf92,311334,9.712437e+15,1,0,0,0,...,0,0,0,0,0,0,0,15.0,1,16.0
11100,1521,NaN,Lusitanian,lcf32,243122,9.712437e+15,0,0,0,0,...,0,0,0,0,0,0,0,12.0,4,25.0
70440,1521,NaN,Lusitanian,NC,322322,9.712437e+15,0,0,0,0,...,0,0,0,0,0,0,0,130.0,55,221.0
79512,1521,10.0,Lusitanian,NC,312312,9.712437e+15,0,0,0,0,...,0,0,0,0,0,0,0,1.0,2,3.0
89194,1521,NaN,Lusitanian,NC,243243,9.712437e+15,183,0,0,0,...,0,0,22,0,0,0,0,1384.0,440,2210.0
94194,1521,NaN,Lusitanian,lcf32,324122,9.712437e+15,0,0,0,0,...,0,0,0,0,0,0,0,35.0,33,95.0
165158,1521,NaN,Lusitanian,lcf37,312133,9.712437e+15,0,0,0,0,...,0,0,0,0,0,0,0,17.0,1,20.0
166747,1521,10.0,Lusitanian,NC,242242,9.712437e+15,0,0,0,0,...,0,0,0,0,0,0,0,23.0,4,27.0
183020,1521,NaN,Lusitanian,lcf37,322133,9.712437e+15,0,0,0,0,...,0,0,0,0,0,0,0,18.0,0,38.0
206981,1521,NaN,Lusitanian,NC,333333,9.712437e+15,266,0,0,0,...,0,0,0,0,0,0,0,562.0,299,1252.0


In [28]:
wildfire[(wildfire['ADM_ID']==1521) & (wildfire['GridNum10km'] == 9712437179711488) & (wildfire['clc0018'] == '321122')]

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01062022,y01072022,y01082022,y01092022,y01102022,y01112022,y01122022,AreaHa,BA_0010,BA_1121
939379,1521,NaN,Lusitanian,lcf32,321122,9.712437e+15,0,0,0,0,...,0,0,0,0,0,0,0,10.0,4,16.0


In [30]:
wildfire[(wildfire['ADM_ID']==1521) & (wildfire['GridNum10km'] == 9712437179711488) & (wildfire['clc0018'] == '321122')][cols_1121].sum(axis=1)

939379    16.0
dtype: float64

In [41]:
cols_1121

Index(['y01012011', 'y01022011', 'y01032011', 'y01042011', 'y01052011',
       'y01062011', 'y01072011', 'y01082011', 'y01092011', 'y01102011',
       ...
       'y01052022', 'y01062022', 'y01072022', 'y01082022', 'y01092022',
       'y01102022', 'y01112022', 'y01122022', 'AreaHa', 'BA_0010'],
      dtype='object', length=146)

In [32]:
t = wildfire[(wildfire['ADM_ID']==1521) & (wildfire['GridNum10km'] == 9712437179711488) & (wildfire['clc0018'] == '321122')][cols_1121].transpose()

In [40]:
t[100:]

,939379
y01052019,0.0
y01062019,0.0
y01072019,0.0
y01082019,0.0
y01092019,0.0
y01102019,0.0
y01112019,0.0
y01122019,0.0
y01012020,0.0
y01022020,0.0


In [9]:
wf_LTA[wf_LTA['ADM_ID']==1521].groupby('ADM_ID').sum()

,ProtArea,GridNum10km,BA_0010,BA_1121,AreaHa,ABA_0010,ABA_1121
ADM_ID,,,,,,,
1521,4333.0,6.792903e+18,28125,149091.0,80649.0,2548.0,13537.0


In [ ]:
wf_LTA[wf_LTA['ADM_ID']==1521].groupby('ADM_ID').sum()

In [13]:
99159+101010

200169

In [10]:
28125+149091

177216

In [11]:
215967-177216

38751

In [17]:
t = wildfire[wildfire['ADM_ID']==1521].groupby('ADM_ID').sum().transpose()

In [20]:
t[t['1521']>0]

KeyError: '1521'

In [3]:
cols_0010 = wildfire.columns[6:138]
wf0010 = wildfire.groupby(['ADM_ID','ProtArea','envZones', 'lcf3', 'clc0018', 'GridNum10km']).sum()
wf0010 = wf0010.reset_index()
wf0010['AreaHa_0010']=wf0010[cols_0010].sum(axis=1)
wf0010.head()

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01052022,y01062022,y01072022,y01082022,y01092022,y01102022,y01112022,y01122022,AreaHa,AreaHa_0010
0,0,1.0,Mediterranean North,NC,211211,2.316892e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,92.0,0
1,0,1.0,Mediterranean North,NC,223223,2.316905e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,25.0,0
2,0,1.0,Mediterranean North,NC,223223,2.316906e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,41.0,0
3,0,1.0,Mediterranean North,NC,242242,2.316899e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,84.0,0
4,0,1.0,Mediterranean North,NC,242242,2.316899e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,6.0,0


In [64]:
cols_1121 = wildfire.columns[138:-1]
wf1121 = wildfire.groupby(['ADM_ID','ProtArea','envZones', 'lcf3', 'clc0018', 'GridNum10km']).sum()
wf1121 = wf1121.reset_index()
wf1121['AreaHa_1121']=wf1121[cols_1121].sum(axis=1)
wf1121.head() 

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01052022,y01062022,y01072022,y01082022,y01092022,y01102022,y01112022,y01122022,AreaHa,AreaHa_1121
0,0,1.0,Mediterranean North,NC,211211,2.316892e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,92.0,0
1,0,1.0,Mediterranean North,NC,223223,2.316905e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,25.0,0
2,0,1.0,Mediterranean North,NC,223223,2.316906e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,41.0,0
3,0,1.0,Mediterranean North,NC,242242,2.316899e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,84.0,0
4,0,1.0,Mediterranean North,NC,242242,2.316899e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,6.0,0


In [ ]:
'''
drop columns with monthly data
wf0010
wf1121
concat or merge
divide total areas / 11 (number of years) LTA long term annual averages

'''

In [ ]:
cols_1121 = wildfire.columns[6:138]
wf0010 = wildfire.groupby(['ADM_ID','GridNum10km']).sum()
wf0010 = wf0010.reset_index()
wf0010['AreaHa_0010']=wf0010[cols_0010].sum(axis=1)
wf0010.head()

In [23]:
t1000 = wildfire.iloc[:, :138]

In [28]:
t1000.head()

,ADM_ID,ProtArea,envZones,lcf3,clc0018,GridNum10km,y01012000,y01022000,y01032000,y01042000,...,y01032010,y01042010,y01052010,y01062010,y01072010,y01082010,y01092010,y01102010,y01112010,y01122010
0,1446,NaN,Boreal,NC,324324,1.930579e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1446,NaN,Boreal,NC,312312,2.364984e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1446,NaN,Boreal,lcf53,412211,2.365057e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1446,NaN,Boreal,NC,313313,2.365072e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1446,NaN,Boreal,NC,313313,2.365080e+16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
cols = t1000.columns[6:138]
print (cols)

Index(['y01012000', 'y01022000', 'y01032000', 'y01042000', 'y01052000',
       'y01062000', 'y01072000', 'y01082000', 'y01092000', 'y01102000',
       ...
       'y01032010', 'y01042010', 'y01052010', 'y01062010', 'y01072010',
       'y01082010', 'y01092010', 'y01102010', 'y01112010', 'y01122010'],
      dtype='object', length=132)


In [51]:
cols_0010 = t1000.columns[6:138]
tsum = t1000.groupby(['ADM_ID','GridNum10km']).sum()
tsum = tsum.reset_index()
tsum

,ADM_ID,GridNum10km,ProtArea,y01012000,y01022000,y01032000,y01042000,y01052000,y01062000,y01072000,...,y01032010,y01042010,y01052010,y01062010,y01072010,y01082010,y01092010,y01102010,y01112010,y01122010
0,6,1.437972e+16,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11,1.889877e+16,11.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11,1.889972e+16,11.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11,1.893367e+16,11.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11,1.893395e+16,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,1604,3.212773e+16,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
975,1606,1.977523e+16,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
976,1606,1.977748e+16,10.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
977,1606,1.979234e+16,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
tsum['total']=tsum[cols].sum(axis=1)

In [53]:
tsum

,ADM_ID,GridNum10km,ProtArea,y01012000,y01022000,y01032000,y01042000,y01052000,y01062000,y01072000,...,y01042010,y01052010,y01062010,y01072010,y01082010,y01092010,y01102010,y01112010,y01122010,total
0,6,1.437972e+16,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11,1.889877e+16,11.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11,1.889972e+16,11.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11,1.893367e+16,11.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11,1.893395e+16,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,1604,3.212773e+16,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
975,1606,1.977523e+16,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
976,1606,1.977748e+16,10.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
977,1606,1.979234e+16,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
tsum['total'].max()

848

In [55]:
tsum[tsum['total']==848]

,ADM_ID,GridNum10km,ProtArea,y01012000,y01022000,y01032000,y01042000,y01052000,y01062000,y01072000,...,y01042010,y01052010,y01062010,y01072010,y01082010,y01092010,y01102010,y01112010,y01122010,total
266,318,2.285937e+16,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,848


TypeError: sum() got an unexpected keyword argument 'axis'

In [41]:
tsum

GridNum10km
0
1
2
3
4
...
995
996
997
998


In [29]:
t1000.groupby(['ADM_ID','GridNum10km'])

In [5]:
df_transformed_AGBflux =df_F_AGBflux_sum.melt(id_vars=["GridNum10km","AreaHa"], 
     var_name="year", 
        value_name="GDMP")
## update date format:
df_transformed_AGBflux['year'] = pd.to_datetime(df_transformed_AGBflux['year'], format='%d-%m-%Y')
print(df_transformed_AGBflux.head(20))

          GridNum10km   AreaHa       year         GDMP
0   22821536360562688  10000.0 2014-11-01          0.0
1    9860785350115328  10000.0 2014-11-01          0.0
2   27660980955643904  10000.0 2014-11-01          0.0
3   18883523796533248   9429.0 2014-11-01          0.0
4   14521696744636416  10000.0 2014-11-01          0.0
5    4842545561468928  10000.0 2014-11-01          0.0
6    5207660731301888  10000.0 2014-11-01          0.0
7   14657941697200128  10000.0 2014-11-01          0.0
8    5144713690611712  10000.0 2014-11-01          0.0
9    9958586050412544  10000.0 2014-11-01          0.0
10  13814440185036800  10000.0 2014-11-01          0.0
11  10176495511142400  10000.0 2014-11-01          0.0
12  27731023282307072  10000.0 2014-11-01          0.0
13   5507681376796672  10000.0 2014-11-01          0.0
14   6021290745921536  10000.0 2014-11-01          0.0
15  27977696139018240  10000.0 2014-11-01   97412981.0
16  14185147402289152  10000.0 2014-11-01  762397386.0
17  189657

### 2. Computing variables

#### 2.1. GDMP anomalies

In [6]:
from  scipy.stats import zscore
def z_score(x):
   z = np.around(zscore(x),1)
   return z

#df['Z_SCORE'] = df.groupby('GROUP')['VALUE'].transform(z_score)
df_transformed_AGBflux['zscore'] = df_transformed_AGBflux.groupby('GridNum10km')['GDMP'].transform(z_score)

In [7]:
GDMP = df_transformed_AGBflux[df_transformed_AGBflux['GDMP']>0]
GDMP.head()

,GridNum10km,AreaHa,year,GDMP,zscore
15,27977696139018240,10000.0,2014-11-01,97412981.0,0.8
16,14185147402289152,10000.0,2014-11-01,762397386.0,1.4
17,18965789600120832,10000.0,2014-11-01,588245574.0,0.3
18,27603260890152960,10000.0,2014-11-01,290296014.0,-1.0
19,9731949216137216,10000.0,2014-11-01,757097776.0,0.3


In [8]:
# create a list of our conditions
conditions = [
    (GDMP['zscore'] <= -1),
    (GDMP['zscore'] > -1) & (GDMP['zscore'] <= 1),
    (GDMP['zscore'] > 1)
    ]
     
# create a list of the values we want to assign for each condition
values = ['-1', '0', '+1']

# create a new column and use np.select to assign values to it using our lists as arguments
GDMP['ANOMALIES'] = np.select(conditions, values)

# display updated DataFrame
GDMP.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,GridNum10km,AreaHa,year,GDMP,zscore,ANOMALIES
15,27977696139018240,10000.0,2014-11-01,97412981.0,0.8,0
16,14185147402289152,10000.0,2014-11-01,762397386.0,1.4,+1
17,18965789600120832,10000.0,2014-11-01,588245574.0,0.3,0
18,27603260890152960,10000.0,2014-11-01,290296014.0,-1.0,-1
19,9731949216137216,10000.0,2014-11-01,757097776.0,0.3,0


##### Export table

In [9]:
name_of_table ="GDMP_Anomalies_10k"
     
print ("connect to greenmonkey engine.....GREENMONKEY.")  
engine = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Climate_Impact" + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
#print ("send table togreenmonkey MS-SQL:")
GDMP.to_sql(name_of_table, engine, if_exists='replace', index = False, schema=None)


connect to greenmonkey engine.....GREENMONKEY.


##### QC GDMP anomalies

In [10]:
print (df_transformed_AGBflux.head())

         GridNum10km   AreaHa       year  GDMP  zscore
0  22821536360562688  10000.0 2014-11-01   0.0     NaN
1   9860785350115328  10000.0 2014-11-01   0.0     NaN
2  27660980955643904  10000.0 2014-11-01   0.0     NaN
3  18883523796533248   9429.0 2014-11-01   0.0     NaN
4  14521696744636416  10000.0 2014-11-01   0.0     NaN


In [11]:
df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 22821536360562688]

,GridNum10km,AreaHa,year,GDMP,zscore
0,22821536360562688,10000.0,2014-11-01,0.0,NaN
302889,22821536360562688,10000.0,2015-11-01,0.0,NaN
605778,22821536360562688,10000.0,2016-11-01,0.0,NaN
908667,22821536360562688,10000.0,2017-11-01,0.0,NaN
1211556,22821536360562688,10000.0,2018-11-01,0.0,NaN
1514445,22821536360562688,10000.0,2019-11-01,0.0,NaN
1817334,22821536360562688,10000.0,2020-11-01,0.0,NaN
2120223,22821536360562688,10000.0,2021-11-01,0.0,NaN


In [12]:
df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 18446609658413056]

,GridNum10km,AreaHa,year,GDMP,zscore
114716,18446609658413056,10000.0,2014-11-01,567420786.0,-0.7
417605,18446609658413056,10000.0,2015-11-01,554946035.0,-1.1
720494,18446609658413056,10000.0,2016-11-01,591739566.0,0.2
1023383,18446609658413056,10000.0,2017-11-01,556189307.0,-1.1
1326272,18446609658413056,10000.0,2018-11-01,626023208.0,1.4
1629161,18446609658413056,10000.0,2019-11-01,597534185.0,0.4
1932050,18446609658413056,10000.0,2020-11-01,634120631.0,1.6
2234939,18446609658413056,10000.0,2021-11-01,565236683.0,-0.7


In [13]:
GDMP_grid1=df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 18446609658413056]
avg = GDMP_grid1['GDMP'].mean()
std = GDMP_grid1['GDMP'].std()
#GDMP_grid2['zscore'] = round(((GDMP_grid2['GDMP']-avg)/std),1)
GDMP_grid1.loc[:,'zscore'] = round(((GDMP_grid1['GDMP']-avg)/std),1)
GDMP_grid1

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,GridNum10km,AreaHa,year,GDMP,zscore
114716,18446609658413056,10000.0,2014-11-01,567420786.0,-0.6
417605,18446609658413056,10000.0,2015-11-01,554946035.0,-1.0
720494,18446609658413056,10000.0,2016-11-01,591739566.0,0.2
1023383,18446609658413056,10000.0,2017-11-01,556189307.0,-1.0
1326272,18446609658413056,10000.0,2018-11-01,626023208.0,1.3
1629161,18446609658413056,10000.0,2019-11-01,597534185.0,0.4
1932050,18446609658413056,10000.0,2020-11-01,634120631.0,1.5
2234939,18446609658413056,10000.0,2021-11-01,565236683.0,-0.7


In [14]:
zsc= stats.zscore(GDMP_grid['GDMP'])
zsc

NameError: name 'GDMP_grid' is not defined

In [ ]:
GDMP_grid2=df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 9731949216137216]
avg = GDMP_grid2['GDMP'].mean()
std = GDMP_grid2['GDMP'].std()
#GDMP_grid2['zscore'] = round(((GDMP_grid2['GDMP']-avg)/std),1)
GDMP_grid2.loc[:,'zscore'] = round(((GDMP_grid2['GDMP']-avg)/std),1)
GDMP_grid2

In [ ]:
GDMP_grid2=df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 9731949216137216]
avg = GDMP_grid2['GDMP'].mean()
std = GDMP_grid2['GDMP'].std()
GDMP_grid2['zscore'] = round(((GDMP_grid2['GDMP']-avg)/std),1)
GDMP_grid2

In [ ]:
GDMP_grid3=df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 27603260890152960]
avg = GDMP_grid3['GDMP'].mean()
std = GDMP_grid3['GDMP'].std()
GDMP_grid3['zscore'] = round(((GDMP_grid3['GDMP']-avg)/std),1)
GDMP_grid3

In [ ]:
avg = GDMP_grid['GDMP'].mean()
std = GDMP_grid['GDMP'].std()
GDMP_grid['zscore'] = round(((GDMP_grid['GDMP']-avg)/std),1)
GDMP_grid

In [ ]:
from  scipy.stats import zscore

In [ ]:
z = np.around(zscore(GDMP_grid1['GDMP']),1)
z

In [ ]:
z = np.round_(zscore(GDMP_grid1['GDMP']),1)
z

In [ ]:
GDMP_grid=df_transformed_AGBflux[(df_transformed_AGBflux['GridNum10km']== 18446609658413056) | (df_transformed_AGBflux['GridNum10km']== 9731949216137216) | (df_transformed_AGBflux['GridNum10km']== 27603260890152960)]
GDMP_grid

In [ ]:
def z_score(x):
   z = np.around(zscore(x),1)
   return z

#df['Z_SCORE'] = df.groupby('GROUP')['VALUE'].transform(z_score)
GDMP_grid['zscore'] = GDMP_grid.groupby('GridNum10km')['GDMP'].transform(z_score)

print (GDMP_grid)

In [ ]:
# create a list of our conditions
conditions = [
    (GDMP_grid['zscore'] <= -1),
    (GDMP_grid['zscore'] > -1) & (GDMP_grid['zscore'] <= 1),
    (GDMP_grid['zscore'] > 1)
    ]
     
# create a list of the values we want to assign for each condition
values = ['-1', '0', '+1']

# create a new column and use np.select to assign values to it using our lists as arguments
GDMP_grid['ANOMALIES'] = np.select(conditions, values)

# display updated DataFrame
GDMP_grid.head()

In [ ]:
GDMP_grid